## Tekst-generering

Her vil vi forsøke å trene et nettverk som skal generere og predikere tekst

In [1]:
from utils import onehot
from data_generators import text_to_training_data
from text_generation import *
from layers import *
from neural_network import NeuralNetwork
import numpy as np

In [2]:
d = 80
n_max = 50
p = 100
k = 25
L = 2

text =  open('treningsTekster/konklusjoner.txt', 'r').read()
data,idx_to_text,text_to_idx, m = text_to_training_data(n_max,text,num_batches=3,batch_size=110)

print("We will train on %d batches of size %d" % (len(data['x_train']),len(data['x_train'][0])))
print("Each sequence has length %d" % n_max)

print("Example of a sequence (chars): \n")
print(''.join([idx_to_text[i] for i in data['x_train'][0][0]]))

print("\nExample of a sequence (idx): \n")
print(data['x_train'][0][0])

data has 17369 characters, 50 unique.
We will train on 3 batches of size 110
Each sequence has length 50
Example of a sequence (chars): 

Basert på våre funn kan vi konkludere med at økend

Example of a sequence (idx): 

[10 23 40 27 39 41  1 38 47  1 43 47 39 27  1 28 42 36 36  1 33 23 36  1
 43 31  1 33 37 36 33 34 42 26 27 39 27  1 35 27 26  1 23 41  1 49 33 27
 36 26]


Vi initialiserer nettverket vårt

In [3]:
embed = EmbedPosition(n_max,m,d)
att1 = Attention(d,k)
att2 = Attention(d,k)
att3 = Attention(d,k)
att4 = Attention(d,k)


ff1 = FeedForward(d,p)
ff2 = FeedForward(d,p)
ff3 = FeedForward(d,p)
ff4 = FeedForward(d,p)

un_embed = LinearLayer(d,m)
softmax = Softmax()

layers = [embed, att1, ff1, att2, ff2, att3, ff3, att4, ff4, un_embed, softmax]

net = NeuralNetwork(layers)
loss = CrossEntropy()

In [4]:
import pickle

#henter vårt ferdigtrente
with open("ferdigtrent_nn", 'rb') as f:
    net = pickle.load(f)

In [5]:
n_iter = 100
batches = 3

x_data = data['x_train']
y_data = data['y_train']


for n in range(n_iter):
    losses = []
    for b in range(batches):
        x = x_data[b]
        y = y_data[b]

        X = onehot(x,m)
        Z = net.forward(X)
        losses.append(loss.forward(Z,y))
        dLdZ = loss.backward()
        net.backward(dLdZ)
        net.step_Adam(0.000005)
    print(np.mean(losses))

0.29197609158509924
0.29191859960720884
0.2918613078656395
0.29180582157369267
0.29174843187394206
0.29169066897891444


KeyboardInterrupt: 

Vi kan også lagre nettverket med biblioteket Pickle

In [ ]:
import pickle

filename = "ferdigtrent"

with open(f"{filename}_nn", 'wb') as f:
    pickle.dump(net, f)

In [ ]:
#We can now generate text from an initial string
start_text = "for å konkludere prosjektet  "
start_idx = np.array([convertToID(start_text,text_to_idx)])
print(start_idx)

[[28 37 39  1 47  1 33 37 36 33 34 42 26 27 39 27  1 38 39 37 40 32 27 33
  41 27 41  1  1]]


In [ ]:
#length of the total text sequence we want to generate
n_gen = n_max*10

generated_idx = generate(net,start_idx,m,n_max,n_gen)

text = convertToTxt(generated_idx[0],idx_to_text)

print(text)

for å konkludere prosjektet  fosormenisklan mamer.
Reltøtere ene sultur sulår ett ensultatsen kr sperenskomar.
Våre for diskon m sn kspludintapplikansern omern ringang væring lvære ekange onsge iljekogrmsme ne t.
Vår Alysyse m audakfor inyster atate hvismendor.
orne potenar dalter rongr iforknypotere rongalsn.
Gjer vålse ar år vår har vår ar arnysertrår alysen re hv pprn ole enarser.
Dennne potuder brudier re menratrinsmmadimmengingir kansimmen dangarnsktwolorsjoniserser hosmer ler inerssten.
vå
